<div class="alert alert-block alert-info">
<h1>1. Общая информация о данных</h1>
    </div>

<ul>
<li>children — количество детей в семье</li>
<li>days_employed — общий трудовой стаж в днях</li>
<li>dob_years — возраст клиента в годах</li>
<li>education — уровень образования клиента</li>
<li>education_id — идентификатор уровня образования</li>
<li>family_status — семейное положение</li>
<li>family_status_id — идентификатор семейного положения</li>
<li>gender — пол клиента</li>
<li>income_type — тип занятости</li>
<li>debt — имел ли задолженность по возврату кредитов</li>
<li>total_income — ежемесячный доход</li>
<li>purpose — цель получения кредита</li>
    </ul>

In [1]:
import pandas as pd
data = pd.read_csv('data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<div class="alert alert-block alert-info">
    <h1> 2. Предобработка данных</h1>
    <h3>1. Избавление от пропущенных значений </h3>
    В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце вы обработаете на следующем этапе. 
Найдите другой столбец и заполните пропущенные значения в нём медианным значением по столбцу:
    <ol>
<li>опишите, какие пропущенные значения вы обнаружили;</li>
<li>проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;</li>
<li>приведите возможные причины появления пропусков в данных;</li>
<li>объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.</li>
    </ol>
</div>

Посмотрим на вывод метода `info`, в нашем случае количество значений в столбце не равно количеству строк в датафрейме для двух столбцов:
1. `days_employed`
2. `total_income`

Посмотрим как выглядят строки с незаполненными `total_income`. Для этого применим логическую индексацию. Запишем отфильтрованные данные в новую переменную для последующего использования.

In [2]:
total_income_empty_entries = data.loc[data['total_income'].isna() == True]
total_income_empty_entries.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


У данных людей есть тип дохода, но нет значения этого дохода. Скорее всего ошибка заполнения данных либо допущенная человеком, либо техническая. Часть информации может быть сокрыта заполняющими умышленно. 

Я бы предложила сгруппировать данные по `income_type` и записать в `total_income` медианное значение. Это позволит посчиать среднее значение зарплаты по группе без учета аномально низких и аномально высоких значений. На мой взгляд значение по группе будет более показательно, чем по всем людям в списке.

In [3]:
ratio = total_income_empty_entries.shape[0] / data.shape[0]
print('Доля пропущенных значений для колонки total income {0:.2%}'.format(ratio))

Доля пропущенных значений для колонки total income 10.10%


Усредненное значение — не самый полезный критерий. Разобьем долю пропусков так же по типу дохода.

In [4]:
total_income_empty = data.loc[data['total_income'].isna() == True].groupby('income_type').count()['purpose']
total_income_empty_ratio = data.loc[data['total_income'].isna() == True].groupby('income_type').count()['purpose'] / data.groupby('income_type').count()['purpose']
total_income_empty_ratio

income_type
безработный             NaN
в декрете               NaN
госслужащий        0.100754
компаньон          0.099902
пенсионер          0.107106
предприниматель    0.500000
сотрудник          0.099379
студент                 NaN
Name: purpose, dtype: float64

Исходя из этой выгрузки мы знаем, что пропущенные данные есть у госслужащих, компаньонов, пенсионеров, предпринимателей и сотрудников. Именно в этих группах мы в последствии сделаем замены. При этом если посмотреть, то 50% предпринемателей не указали `total_income`. Либо предпринимателей очень мало, либо они активно скрывают свои доходы.

In [5]:
data.groupby('income_type')['total_income'].count()

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: total_income, dtype: int64

По данным о количестве людей в группах "безработный", "в декрете", "предприниматель", "студент" понятно, что в этих группах не важно использовать медианное или среднее арифметическое значение. Это связано с тем, что данных меньше трех. В этих случаях оба средних равны между собой.

Посчитаем среднее арифметическое и среднее медианное, минимум и максимум значения дохода по типу дохода, это поможет принять окончательное решение заполнять пропущенные данные средним арифметическим или средним медианным значением.

In [6]:
groups_for_update = ('госслужащий', 'компаньон', 'пенсионер', 'сотрудник', 'предприниматель')
only_stats_data = data.loc[data['income_type'].isin(groups_for_update)]

In [7]:
only_stats_data.groupby('income_type')['total_income'].min()

income_type
госслужащий         29200.077193
компаньон           28702.812889
пенсионер           20667.263793
предприниматель    499163.144947
сотрудник           21367.648356
Name: total_income, dtype: float64

In [8]:
# Избавимся от экспоненциальной записи слишком больших чисел изменив настройки отображения
pd.options.display.float_format = '{:.2f}'.format

only_stats_data.groupby('income_type')['total_income'].max()

income_type
госслужащий        910451.47
компаньон         2265604.03
пенсионер          735103.27
предприниматель    499163.14
сотрудник         1726276.01
Name: total_income, dtype: float64

In [9]:
mean_total_income_by_income_type = only_stats_data.groupby('income_type')['total_income'].mean()
mean_total_income_by_income_type

income_type
госслужащий       170898.31
компаньон         202417.46
пенсионер         137127.47
предприниматель   499163.14
сотрудник         161380.26
Name: total_income, dtype: float64

In [10]:
median_total_income_by_income_type = only_stats_data.groupby('income_type')['total_income'].median()
median_total_income_by_income_type

income_type
госслужащий       150447.94
компаньон         172357.95
пенсионер         118514.49
предприниматель   499163.14
сотрудник         142594.40
Name: total_income, dtype: float64

Разница между средним арифметическим и медианным показывает, что во всех группах есть небольшой процент людей получающий ощутимо больше других. Для статистики такие аномалии вредны,

Сделаем замены:
Для каждой группы по типу доходов найдем все записи для которых общий доход не указан и запишем туда значение из serias с медианными значениями по выбранной группе. Для реализации такого алгоритма потребуется цикл, логическая индексация и обычная индексация.

In [11]:
income_types_with_empty = ('госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник')
for income_type in income_types_with_empty:
    data.loc[(data['income_type'] == income_type) & (data['total_income'].isna() == True), 'total_income'] = median_total_income_by_income_type.loc[income_type]
    

In [12]:
data.loc[data['total_income'].isna() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Все пропуски заполнены. Можно переходить к следующему этапу

<div class="alert alert-block alert-info">
<h3> 2. Исправление аномалии </h3> 
В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных. После обработки аномалий заполните пропуски в days_employed медианными значениями по этому столбцу.
</div>

In [13]:
print('Do it later')

Do it later


In [14]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [15]:
data.loc[data['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.22,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.87,покупка жилья
720,20,-855.60,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.74,покупка недвижимости
1074,20,-3310.41,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.54,получение образования
2510,20,-2714.16,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.84,операции с коммерческой недвижимостью
2941,20,-2161.59,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.94,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.26,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.01,свой автомобиль
21325,20,-601.17,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.07,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,172357.95,покупка жилой недвижимости
21404,20,-494.79,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.68,операции со своей недвижимостью


In [16]:
data.loc[data['children'].isin([-1, 20]), 'children'] = 0
data['children'].value_counts().sort_index()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [17]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [18]:
data[data['gender'] == 'XNA']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.60,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.16,покупка недвижимости


Посмотрев на строку с данными нельзя сделать однозначный вывод про пол. Простой выбор -- удалить строку с некорректными данными.

In [19]:
data = data.drop(data[data['gender'] == 'XNA'].index).reset_index(drop=True)
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

<div class="alert alert-block alert-info">
    <h3>3. Исправление типов данных </h3>
Замените вещественный тип данных в столбце total_income на целочисленный, например, с помощью метода astype().
    </div>

In [20]:
try:
    data = data.astype({'total_income': 'int'})
except:
    print('Возникла ошибка преобразования. Хотя казалось бы каким образом из float в int может возникнуть ошибка.')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  int64  
 11  purpose           21524 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


<div class="alert alert-block alert-info">
<h3>4. Удаление дубликатов</h3> 
Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к одному регистру. Проверьте остальные столбцы.
После удаления дубликатов сделайте следующее:
    <ol>
<li>поясните, как выбирали метод для поиска и удаления дубликатов в данных;</li>
<li>приведите возможные причины появления дубликатов.</li>
    </ol>
    </div>

In [21]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [22]:
print('Количество полных дубликатов:', data.duplicated().sum())

Количество полных дубликатов: 54


In [23]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [24]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [25]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Собрала несколько вольную группировку что во что переименовать. Есть сомнения с группировкой "жилье" в обычной жизни пошла бы к поставщикам данных уточнить когда что пишут


In [26]:
purposes_rename_dict = {
    'свадьба':  ['на проведение свадьбы', 'сыграть свадьбу'],
    'коммерческая недвижимость': ['покупка коммерческой недвижимости', 'покупка жилья для сдачи', 'операции с жильем', 'операции с коммерческой недвижимостью'],
    'жилье': ['покупка жилья', 'покупка жилья для семьи', 'покупка своего жилья', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости', 'недвижимость', 'строительство недвижимости', 'ремонт жилью', 'операции с недвижимостью'],              
    'автомобиль': ['сделка с подержанным автомобилем', 'свой автомобиль', 'на покупку подержанного автомобиля', 'автомобили', 'на покупку автомобиля', 'приобретение автомобиля', 'сделка с автомобилем', 'на покупку своего автомобиля'],
    'образование': ['высшее образование', 'получение высшего образования', 'заняться образованием', 'профильное образование', 'получение образования', 'получение дополнительного образования', 'дополнительное образование', 'заняться высшим образованием'],
}
for final_name, bad_names in purposes_rename_dict.items():
    for name in bad_names:
        data.loc[data['purpose'] == name, 'purpose'] = final_name
        

In [27]:
data['purpose'].value_counts()

жилье                        8218
автомобиль                   4315
образование                  4022
коммерческая недвижимость    2621
свадьба                      2348
Name: purpose, dtype: int64

In [28]:
median_age = int(data.loc[data['dob_years'] != 0, 'dob_years'].median())
median_age

43

In [29]:
data.loc[(data['dob_years'] == 0), 'dob_years'] = median_age
data['dob_years'].value_counts().sort_index().head()

19     14
20     51
21    111
22    183
23    254
Name: dob_years, dtype: int64

<div class="alert alert-block alert-info">
<h3> 5. Выделение избыточных значений </h3> 
Создайте два новых датафрейма, в которых:
    <ol>
<li>каждому уникальному значению из education соответствует уникальное значение education_id — в первом;</li>
<li>каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.</li>
    </ol>
Удалите из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. Новые датафреймы — это те самые «словари» (не путайте с одноимённой структурой данных в Python), к которым вы сможете обращаться по идентификатору.
    </div>

In [30]:
education_dict = data.loc[:, ('education_id', 'education')].drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [31]:
family_status_dict = data.loc[:, ('family_status_id', 'family_status')].drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [32]:
data = data.drop(['education', 'family_status'], axis=1)
data.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

In [33]:
print('Количество дубликатов после нормализации данных:', data.duplicated().sum())
data=data.drop_duplicates().reset_index(drop=True)
print('Проверка успешности зачистки дубликатов:', data.duplicated().sum())

Количество дубликатов после нормализации данных: 335
Проверка успешности зачистки дубликатов: 0


<div class="alert alert-block alert-info">
    <h3>6. Категоризация суммарного дохода</h3>
    На основании диапазонов, указанных ниже, создайте столбец total_income_category с категориями:
    <ul>
<li>0–30000 — 'E';</li>
<li>30001–50000 — 'D';</li>
<li>50001–200000 — 'C';</li>
<li>200001–1000000 — 'B';</li>
<li>1000001 и выше — 'A'</li>
    </ul>
Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.
    </div>

In [34]:
def get_total_income_category(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    if 30000 < total_income <= 50000:
        return 'D'
    if 50000 < total_income <= 200000:
        return 'C'
    if 200000 < total_income <= 1000000:
        return 'B'
    if 1000000 < total_income:
        return 'A'
    return 'ERROR: negative income'

In [35]:
data['total_income_category'] = data['total_income'].apply(get_total_income_category)
data['total_income_category'].value_counts()

C    15751
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

<div class="alert alert-block alert-info">
    <h3>7. Категоризация целей</h3>
    Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
    <ul>
<li>'операции с автомобилем',</li>
<li>'операции с недвижимостью',</li>
<li>'проведение свадьбы',</li>
<li>'получение образования'</li>
    </ul>
Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.
Вы можете использовать собственную функцию и метод apply(). Изучите данные в столбце purpose и определите, какие подстроки помогут вам правильно определить категорию.
    </div>

In [36]:
data['purpose'].value_counts()

жилье                        8041
автомобиль                   4271
образование                  3964
коммерческая недвижимость    2607
свадьба                      2306
Name: purpose, dtype: int64

In [37]:
def get_purpose_category(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'образовани' in purpose:
        return 'получение образования'
    if 'недвижимост' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    return 'ERROR unknown purpose category'

In [38]:
data['purpose_category'] = data['purpose'].apply(get_purpose_category)
data['purpose_category'].value_counts()

операции с недвижимостью    10648
операции с автомобилем       4271
получение образования        3964
проведение свадьбы           2306
Name: purpose_category, dtype: int64

<div class="alert alert-block alert-info">
<h1>3. Ответы на вопросы</h1>
</div>

<div class="alert alert-block alert-info">
    <h3> 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?</h3>
</div>

In [39]:
data.loc[:,['children', 'debt']].pivot_table(index='children', values='debt', aggfunc=('sum', 'count', 'mean')).reset_index()

,children,count,mean,sum
0,0,13999.00,0.08,1070.00
1,1,4768.00,0.09,444.00
2,2,2044.00,0.09,194.00
3,3,329.00,0.08,27.00
4,4,40.00,0.10,4.00
5,5,9.00,0.00,0.00


Выводы: с большей вероятностью кредит выплачивают без задолжностей люди без детей либо с тремя детьми. Всего 8% просрочек.
Люди с 5 детьми не имели никаких задолжностей, однако, на мой взгляд нельзя опираться на эти данные так как объем выборки всего 9 человек.
Хуже всего кредит выплачивают люди с 4 детьми. В их случае процент просрочек - 10%

<div class="alert alert-block alert-info">
    <h3>2. Есть ли зависимость между семейным положением и возвратом кредита в срок?</h3>
   </div>

In [40]:
data.loc[:,['family_status_id', 'debt']].pivot_table(index='family_status_id', values='debt', aggfunc=('sum', 'count', 'mean')).merge(family_status_dict, on='family_status_id')

,family_status_id,count,mean,sum,family_status
0,0,12133.00,0.08,929.00,женат / замужем
1,1,4127.00,0.09,388.00,гражданский брак
2,2,944.00,0.07,63.00,вдовец / вдова
3,3,1193.00,0.07,85.00,в разводе
4,4,2792.00,0.10,274.00,Не женат / не замужем


<div class="alert alert-block alert-info">
    <h3>3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</h3>
    </div>

In [41]:
data.loc[:,['total_income_category', 'debt']].pivot_table(index='total_income_category', values='debt', aggfunc=('sum', 'count', 'mean')).reset_index()

,total_income_category,count,mean,sum
0,A,25.00,0.08,2.00
1,B,5041.00,0.07,356.00
2,C,15751.00,0.09,1358.00
3,D,350.00,0.06,21.00
4,E,22.00,0.09,2.00


<div class="alert alert-block alert-info">
    <h3>4. Как разные цели кредита влияют на его возврат в срок?</h3>
    </div>

In [42]:
data.loc[:,['purpose_category', 'debt']].pivot_table(index='purpose_category', values='debt', aggfunc=('sum', 'count', 'mean')).reset_index()

,purpose_category,count,mean,sum
0,операции с автомобилем,4271.00,0.09,402.00
1,операции с недвижимостью,10648.00,0.07,781.00
2,получение образования,3964.00,0.09,370.00
3,проведение свадьбы,2306.00,0.08,186.00


<div class="alert alert-block alert-info">
<h1>Общий вывод</h1>
</div>